In [1]:
import csv
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import NoSuchElementException

from datetime import datetime

In [ ]:
attractions = ['https://www.tripadvisor.co.id/Attraction_Review-g294264-d2439664-Reviews-Universal_Studios_Singapore-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d4009739-Reviews-S_E_A_Aquarium-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d3747640-Reviews-Adventure_Cove_Waterpark-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d3475270-Reviews-Sentosa_Boardwalk-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d7178019-Reviews-Madame_Tussauds_Singapore-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1997266-Reviews-Singapore_Cable_Car_Sentosa-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d941828-Reviews-Palawan_Beach-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1626904-Reviews-Skyline_Luge_Sentosa-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d6714698-Reviews-Trick_Eye_Museum_Singapore-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1758910-Reviews-Mega_Adventure-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d2180413-Reviews-IFly_Singapore-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1936430-Reviews-Sentosa_4D_Adventureland-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294265-d12795233-Reviews-Skypark_Sentosa_by_AJ_Hackett-Singapore.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d19944077-Reviews-HydroDash-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1891492-Reviews-Siloso_Beach-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d324747-Reviews-Fort_Siloso-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1891491-Reviews-Tanjong_Beach-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1997267-Reviews-Sentosa_Nature_Discovery-Sentosa_Island.html']

## Version 2
- Able to count number of native reviews
- Able to scrape and write to csv file

Next:
- Expand reviews

In [ ]:
# Version 2 (Add page counter)

def scrape_review_v2(attraction, link):
  
  start_time = datetime.now()
  
  # Open csv file to write in
  file_path = f'data/raw/{attraction}.csv'
  file = open(file_path, 'a', encoding="utf-8", newline='')
  csv_writer = csv.writer(file)

  # Open up browser and navigate to page
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
  driver.get(link)
  
  # Get number of pages
  total_reviews = driver.find_element(By.XPATH, "//span[@class='cfIVb'][normalize-space()='6.744']").text.replace('.', '')
  total_reviews = int(total_reviews)
  num_pages = (total_reviews // 10) + 1
  
  # Date replace
  date_dict = {'Okt':'Oct', 'Des':'Dec', 'Agt':'Aug', 'Mei':'May'}
  
  # Set review and page number counter
  review_count = 0
  page_num = 1
  
  for i in range(num_pages):
    
    # Let page load (change to until element exist later)
    time.sleep(3)
    
    # Extract elements
    container = driver.find_elements(By.XPATH, "//*[@data-automation='reviewCard']//span[contains(text(),'Google Terjemahan')]")      
    
    # Extract elements               
    reviews = driver.find_elements(By.XPATH, "//div[@data-automation='reviewCard']/div[5]/div[1]")
    dates = driver.find_elements(By.XPATH, "//*[@class='eRduX']")
    
    # Open csv file and write to it
    for j in range(len(reviews)):
      review = reviews[j].text.replace("\n", " ").replace("\t", " ")
      date = dates[j].text[:9].strip()
      
      # Replace date with proper abbreviations
      for key, val in date_dict.items():
        if date[:3] == key:
          date = date.replace(key, val)
      
      # Write reviews and date into csv file
      csv_writer.writerow([date, review])      
    
    review_count += len(container)
    
    # Click next page
    next = driver.find_element(By.XPATH, "//a[@aria-label='Next page']//*[name()='svg']")
    next.click()
    page_num += 1
    
    # If no. of native reviews on page is 0, exit loop
    if len(container) == 0:
      break
    
    print(f"Review Count: {review_count}")
    print(f"Current page: {page_num}")
  
  print('-------------- Scrape Completed --------------')
  print(f"Total Native Reviews: {review_count}")
  end_time = datetime.now()
  print('Runtime: {}'.format(end_time - start_time))
  
  file.close()
  # driver.close()

In [ ]:
scrape_review_v2('sea_aquarium', 'https://www.tripadvisor.co.id/Attraction_Review-g294264-d4009739-Reviews-S_E_A_Aquarium-Sentosa_Island.html')

In [2]:
# Version 3

def scrape_review_v3(attraction, link):
  
  start_time = datetime.now()
  
  # Open csv file to write in
  file_path = f'data/raw/{attraction}.csv'
  file = open(file_path, 'a', encoding="utf-8", newline='')
  csv_writer = csv.writer(file)

  # Open up browser and navigate to page
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
  driver.get(link)
  
  # Get number of pages
  total_reviews = driver.find_element(By.XPATH, "//span[@class='cfIVb'][normalize-space()='6.744']").text.replace('.', '')
  total_reviews = int(total_reviews)
  num_pages = (total_reviews // 10) + 1
  
  # Date replace
  date_dict = {'Okt':'Oct', 'Des':'Dec', 'Agt':'Aug', 'Mei':'May'}
  
  # Set review and page number counter
  review_count = 0
  page_num = 1
  
  for i in range(num_pages):
    
    # Let page load (change to until element exist later)
    time.sleep(3)      
    
    # Extract elements               
    reviews = driver.find_elements(By.XPATH, "//span[contains(text(),'Google Terjemahan')]//parent::button//parent::div//div[position()=5]/div[1]/div[1]")
    # dates = driver.find_elements(By.XPATH, "//*[@class='eRduX']")
    
    # Open csv file and write to it
    for j in range(len(reviews)):
      
      review = reviews[j].text.replace('<br>', '')
      review = review.replace('</br>', '')
      # date = dates[j].text[:9].strip()
      
      # Replace date with proper abbreviations
      # for key, val in date_dict.items():
      #   if date[:3] == key:
      #     date = date.replace(key, val)
      
      # Write reviews and date into csv file
      csv_writer.writerow([review])      
      
    review_count += len(reviews)
    
    # Click next page
    next = driver.find_element(By.XPATH, "//a[@aria-label='Next page']//*[name()='svg']")
    next.click()
    page_num += 1
    
    # If no. of native reviews on page is 0, exit loop
    if len(reviews) == 0:
      break
    
    print(f"Review Count: {review_count}")
    print(f"Current page: {page_num}")
  
  print('-------------- Scrape Completed --------------')
  print(f"Total Native Reviews: {review_count}")
  end_time = datetime.now()
  print('Runtime: {}'.format(end_time - start_time))
  
  file.close()
  # driver.close()

In [3]:
scrape_review_v3('sea_aquarium', 'https://www.tripadvisor.co.id/Attraction_Review-g294264-d4009739-Reviews-S_E_A_Aquarium-Sentosa_Island.html')



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Luqman\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


Review Count: 10
Current page: 2
Review Count: 20
Current page: 3
Review Count: 30
Current page: 4
Review Count: 40
Current page: 5
Review Count: 50
Current page: 6
Review Count: 60
Current page: 7
Review Count: 70
Current page: 8
Review Count: 80
Current page: 9
Review Count: 90
Current page: 10
Review Count: 100
Current page: 11
Review Count: 110
Current page: 12
Review Count: 119
Current page: 13
Review Count: 129
Current page: 14
Review Count: 138
Current page: 15
Review Count: 141
Current page: 16
-------------- Scrape Completed --------------
Total Native Reviews: 141
Runtime: 0:00:55.876877
